In [ ]:
# 本文件以流计算的思路，依次处理每一条日志记录，查找确切的用户历史行为日志

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
dataDir = '../data'

### 查找历史问题和回答

In [30]:
invite_df= pd.read_csv(os.path.join(dataDir, 'invite_info_1021.csv'),
                # columns = ['question_id', 'user_id', 'create_time', 'create_day', 'create_hour', 'isAnswer']
                      sep= '\t',
                       index_col= 0,
                     )

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [31]:
invite_df.columns

Index(['question_id', 'user_id', 'create_time', 'create_day', 'create_hour',
       'is_answer'],
      dtype='object')

In [21]:
test_df = pd.read_csv(os.path.join(dataDir, 'invite_info_evaluate_1_0926.txt'), 
                     names = ['question_id', 'user_id', 'create_time'],
                     sep= '\t')

In [24]:
test_df.head()

,question_id,user_id,create_time
0,Q1493039281,M64135255,D3870-H9
1,Q2023398782,M2536956560,D3872-H22
2,Q4151338694,M3294926344,D3874-H15
3,Q3271436624,M3744310794,D3873-H4
4,Q3314287018,M1349051752,D3872-H19


In [25]:
test_df['create_day'] = test_df['create_time'].apply(lambda x: int(x.split('-')[0].lstrip('D')))

In [26]:
test_df['create_hour'] = test_df['create_time'].apply(lambda x: int(x.split('-')[1].lstrip('H')))

In [ ]:
test_df['test_order']

In [27]:
test_df.head()

,question_id,user_id,create_time,create_day,create_hour
0,Q1493039281,M64135255,D3870-H9,3870,9
1,Q2023398782,M2536956560,D3872-H22,3872,22
2,Q4151338694,M3294926344,D3874-H15,3874,15
3,Q3271436624,M3744310794,D3873-H4,3873,4
4,Q3314287018,M1349051752,D3872-H19,3872,19


In [28]:
test_df.to_csv(os.path.join(dataDir, 'invite_info_evaluate_1021.csv'), sep= '\t')

In [32]:
test_df['is_test'] = 1

In [33]:
test_df['is_answer']= 0

In [34]:
test_df.head()

,question_id,user_id,create_time,create_day,create_hour,is_test,is_answer
0,Q1493039281,M64135255,D3870-H9,3870,9,1,0
1,Q2023398782,M2536956560,D3872-H22,3872,22,1,0
2,Q4151338694,M3294926344,D3874-H15,3874,15,1,0
3,Q3271436624,M3744310794,D3873-H4,3873,4,1,0
4,Q3314287018,M1349051752,D3872-H19,3872,19,1,0


In [35]:
invite_df['is_test']= 0

In [36]:
full_invite_df= pd.concat([invite_df, test_df], axis= 0)

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [37]:
full_invite_df.head()

,create_day,create_hour,create_time,is_answer,is_test,question_id,user_id
0,3865,22,D3865-H22,0,0,Q2166419046,M401693808
1,3844,11,D3844-H11,0,0,Q1550017551,M3392373099
2,3862,15,D3862-H15,0,0,Q604029601,M2317670257
3,3849,11,D3849-H11,0,0,Q2350061229,M1618461867
4,3867,4,D3867-H4,0,0,Q2443223942,M3544409350


In [46]:
ans_df = pd.read_csv(os.path.join(dataDir, 'answer_info_1106.csv'), sep= '\t', usecols = ['answer_id', 'question_id', 'user_id'], 
                     # index_col= 'answer_id',
                    nrows= None)

In [47]:
ans_df.head()

,answer_id,question_id,user_id
0,A2502060945,Q1867533817,M625498202
1,A2847829478,Q3366788616,M142330444
2,A2005999231,Q4264694221,M771499642
3,A14821523,Q1088851650,M2282072267
4,A731550034,Q1023877868,M2282072267


In [48]:
test_invite_df = pd.merge(full_invite_df, ans_df, on = ['question_id', 'user_id'], how= 'left')

In [49]:
ans_found = np.logical_xor(test_invite_df['is_answer'] == 1, test_invite_df['answer_id'].isna())

In [50]:
test_invite_df = test_invite_df.loc[ans_found]

In [52]:
len(ans_found), np.sum(ans_found)

(10630845, 10448262)

#### 大约有十几万个邀请记录得到回答，却没有回答信息

#### 寻找历史 question index , answer index

In [53]:
max_hist_len = 32

In [54]:
test_invite_df.head()

,create_day,create_hour,create_time,is_answer,is_test,question_id,user_id,answer_id
0,3865,22,D3865-H22,0,0,Q2166419046,M401693808,NaN
1,3844,11,D3844-H11,0,0,Q1550017551,M3392373099,NaN
2,3862,15,D3862-H15,0,0,Q604029601,M2317670257,NaN
3,3849,11,D3849-H11,0,0,Q2350061229,M1618461867,NaN
4,3867,4,D3867-H4,0,0,Q2443223942,M3544409350,NaN


In [55]:
test_invite_df = test_invite_df.sort_values(by= ['user_id', 'create_day', 'create_hour'])

In [56]:
test_invite_df.head()

,create_day,create_hour,create_time,is_answer,is_test,question_id,user_id,answer_id
8239342,3842,16,D3842-H16,1,0,Q2574396618,M1000000382,A1260717197
5345128,3844,15,D3844-H15,0,0,Q2557202265,M1000000382,NaN
9167807,3846,21,D3846-H21,0,0,Q3910864809,M1000000382,NaN
3507059,3860,21,D3860-H21,0,0,Q1282394327,M1000000382,NaN
736806,3861,17,D3861-H17,0,0,Q4251023727,M1000000382,NaN


In [58]:
days_since_last_ans = []
hist_ans_ids = []
hist_quest_ids = []
user_hist_quest = []
user_hist_ans = []
cur_user_id = 'M1000000382'
last_ans_day = 3830
for i, row in enumerate(test_invite_df.itertuples()):
    if row.user_id == cur_user_id:
        days_since_last_ans.append(row.create_day - last_ans_day)
        hist_ans_ids.append(user_hist_ans + ['-1'] * (max_hist_len - len(user_hist_ans)))
        hist_quest_ids.append(user_hist_quest + ['-1'] * (max_hist_len - len(user_hist_quest)))
        if row.is_answer == 1:
            last_ans_day = row.create_day
            if len(user_hist_ans) < max_hist_len:
                user_hist_ans.append(row.answer_id)
                user_hist_quest.append(row.question_id)
    else:
        last_ans_day = 3830
        days_since_last_ans.append(row.create_day - last_ans_day)
        user_hist_ans = []
        user_hist_quest = []
#         hist_quest_id_df.iloc[i, :len(user_hist_ans)] = user_hist_ans
        hist_ans_ids.append(['-1'] * max_hist_len)
        hist_quest_ids.append(['-1'] * max_hist_len)
        cur_user_id = row.user_id
    if i % 100000 == 0:
        print('%d / %d' %(i, len(test_invite_df)))

0 / 10448262
100000 / 10448262
200000 / 10448262
300000 / 10448262
400000 / 10448262
500000 / 10448262
600000 / 10448262
700000 / 10448262
800000 / 10448262
900000 / 10448262
1000000 / 10448262
1100000 / 10448262
1200000 / 10448262
1300000 / 10448262
1400000 / 10448262
1500000 / 10448262
1600000 / 10448262
1700000 / 10448262
1800000 / 10448262
1900000 / 10448262
2000000 / 10448262
2100000 / 10448262
2200000 / 10448262
2300000 / 10448262
2400000 / 10448262
2500000 / 10448262
2600000 / 10448262
2700000 / 10448262
2800000 / 10448262
2900000 / 10448262
3000000 / 10448262
3100000 / 10448262
3200000 / 10448262
3300000 / 10448262
3400000 / 10448262
3500000 / 10448262
3600000 / 10448262
3700000 / 10448262
3800000 / 10448262
3900000 / 10448262
4000000 / 10448262
4100000 / 10448262
4200000 / 10448262
4300000 / 10448262
4400000 / 10448262
4500000 / 10448262
4600000 / 10448262
4700000 / 10448262
4800000 / 10448262
4900000 / 10448262
5000000 / 10448262
5100000 / 10448262
5200000 / 10448262
5300000 

In [59]:
test_invite_df['days_since_last_ans'] = days_since_last_ans

In [60]:
test_invite_df['days_since_last_ans'].describe()

count    1.044826e+07
mean     1.581494e+01
std      1.179044e+01
min      0.000000e+00
25%      6.000000e+00
50%      1.400000e+01
75%      2.400000e+01
max      4.400000e+01
Name: days_since_last_ans, dtype: float64

In [63]:
from sklearn.preprocessing import MinMaxScaler

In [64]:
days_since_las_scaler = MinMaxScaler((0, 2))

In [65]:
test_invite_df['days_since_last_ans_scaled'] = days_since_las_scaler.fit_transform(test_invite_df[['days_since_last_ans', ]])

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [104]:
test_invite_df['create_weekday'] = test_invite_df['create_day'] % 7

In [105]:
test_invite_df.head()

,create_day,create_hour,create_time,is_answer,is_test,question_id,user_id,answer_id,days_since_last_ans,days_since_last_ans_scaled,create_weekday
8239342,3842,16,D3842-H16,1,0,Q2574396618,M1000000382,A1260717197,12,0.545455,6
5345128,3844,15,D3844-H15,0,0,Q2557202265,M1000000382,NaN,2,0.090909,1
9167807,3846,21,D3846-H21,0,0,Q3910864809,M1000000382,NaN,4,0.181818,3
3507059,3860,21,D3860-H21,0,0,Q1282394327,M1000000382,NaN,18,0.818182,3
736806,3861,17,D3861-H17,0,0,Q4251023727,M1000000382,NaN,19,0.863636,4


In [106]:
train_invite_df = test_invite_df.loc[test_invite_df['is_test'] == 0]

In [107]:
new_invite_df = test_invite_df.loc[test_invite_df['is_test'] == 1]

In [108]:
train_invite_df.to_csv(os.path.join(dataDir, 'invite_info_1107.csv'), sep= '\t')

In [109]:
new_invite_df.to_csv(os.path.join(dataDir, 'test_invite_info_1107.csv'), sep= '\t')

In [76]:
quest_id2ind = pd.read_csv(os.path.join(dataDir, 'question_info_1106.csv'),
                           usecols= ['question_id', 'question_index'],
                           sep= '\t',
                          index_col= 'question_id',
                          squeeze= True).to_dict()

In [77]:
quest_id2ind['-1'] = -1

In [78]:
hist_quest_inds = []
for i, ls in enumerate(hist_quest_ids):
    hist_quest_inds.append([quest_id2ind[quest_id] for quest_id in ls])
    if i % 100000 == 0:
        print( '%d / %d' %(i, len(hist_quest_ids)))

0 / 10448262
100000 / 10448262
200000 / 10448262
300000 / 10448262
400000 / 10448262
500000 / 10448262
600000 / 10448262
700000 / 10448262
800000 / 10448262
900000 / 10448262
1000000 / 10448262
1100000 / 10448262
1200000 / 10448262
1300000 / 10448262
1400000 / 10448262
1500000 / 10448262
1600000 / 10448262
1700000 / 10448262
1800000 / 10448262
1900000 / 10448262
2000000 / 10448262
2100000 / 10448262
2200000 / 10448262
2300000 / 10448262
2400000 / 10448262
2500000 / 10448262
2600000 / 10448262
2700000 / 10448262
2800000 / 10448262
2900000 / 10448262
3000000 / 10448262
3100000 / 10448262
3200000 / 10448262
3300000 / 10448262
3400000 / 10448262
3500000 / 10448262
3600000 / 10448262
3700000 / 10448262
3800000 / 10448262
3900000 / 10448262
4000000 / 10448262
4100000 / 10448262
4200000 / 10448262
4300000 / 10448262
4400000 / 10448262
4500000 / 10448262
4600000 / 10448262
4700000 / 10448262
4800000 / 10448262
4900000 / 10448262
5000000 / 10448262
5100000 / 10448262
5200000 / 10448262
5300000 

In [79]:
hist_quest_ind_array = np.array(hist_quest_inds)

In [80]:
train_hist_quest_array = hist_quest_ind_array[test_invite_df['is_test'] == 0]

In [81]:
test_hist_quest_array = hist_quest_ind_array[test_invite_df['is_test'] == 1]

In [82]:
train_hist_quest_array.shape

(9306579, 32)

In [83]:
test_hist_quest_array.shape

(1141683, 32)

In [85]:
np.save(os.path.join(dataDir, 'train_hist_quest_array.npy'),  train_hist_quest_array)

In [93]:
np.save(os.path.join(dataDir, 'test_hist_quest_array.npy'),  test_hist_quest_array)

In [87]:
ans_id2ind = pd.read_csv(os.path.join(dataDir, 'answer_info_1106.csv'),
                           usecols= ['answer_id', 'answer_index'],
                           sep= '\t',
                          index_col= 'answer_id',
                          squeeze= True).to_dict()

In [88]:
ans_id2ind['-1'] = -1

In [89]:
hist_ans_inds = []
for i, ls in enumerate(hist_ans_ids):
    hist_ans_inds.append([ans_id2ind[ans_id] for ans_id in ls])
    if i % 100000 == 0:
        print( '%d / %d' %(i, len(hist_ans_ids)))

0 / 10448262
100000 / 10448262
200000 / 10448262
300000 / 10448262
400000 / 10448262
500000 / 10448262
600000 / 10448262
700000 / 10448262
800000 / 10448262
900000 / 10448262
1000000 / 10448262
1100000 / 10448262
1200000 / 10448262
1300000 / 10448262
1400000 / 10448262
1500000 / 10448262
1600000 / 10448262
1700000 / 10448262
1800000 / 10448262
1900000 / 10448262
2000000 / 10448262
2100000 / 10448262
2200000 / 10448262
2300000 / 10448262
2400000 / 10448262
2500000 / 10448262
2600000 / 10448262
2700000 / 10448262
2800000 / 10448262
2900000 / 10448262
3000000 / 10448262
3100000 / 10448262
3200000 / 10448262
3300000 / 10448262
3400000 / 10448262
3500000 / 10448262
3600000 / 10448262
3700000 / 10448262
3800000 / 10448262
3900000 / 10448262
4000000 / 10448262
4100000 / 10448262
4200000 / 10448262
4300000 / 10448262
4400000 / 10448262
4500000 / 10448262
4600000 / 10448262
4700000 / 10448262
4800000 / 10448262
4900000 / 10448262
5000000 / 10448262
5100000 / 10448262
5200000 / 10448262
5300000 

In [90]:
hist_ans_ind_array = np.array(hist_ans_inds)

In [91]:
train_hist_ans_array = hist_ans_ind_array[test_invite_df['is_test'] == 0]

In [92]:
test_hist_ans_array = hist_ans_ind_array[test_invite_df['is_test'] == 1]

In [94]:
train_hist_ans_array.shape

(9306579, 32)

In [95]:
test_hist_ans_array.shape

(1141683, 32)

In [96]:
np.save(os.path.join(dataDir, 'train_hist_ans_array.npy'),  train_hist_ans_array)

In [97]:
np.save(os.path.join(dataDir, 'test_hist_ans_array.npy'),  test_hist_ans_array)

In [100]:
train_hist_ans_array[:10,]

array([[    -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1],
       [860889,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1],
       [860889,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -1,     -1],
       [860889,     -1,     -1,     -1,     -1,     -1,     -1,     -1,
            -1,     -1,     -1,     -1,     -1,     -1,     -

#### 邀请距问题创建的天数